2. SQL Challenge (Use SQL for this section)

The business would like to understand what the overlap is between 2018 payment totals for their biggest clients and the rank of the overall payment totals within each entity. Note defaults do not count as payments.

Please provide a csv containing the client IDs for 2018's top 20 clients when sorted on payment amount, the entity type, 2018's total payment amount, and their overall (across all years) payment amount rank within the entity they belong to. Also include your SQL code.

In [1]:
import pandas as pd
from pandasql import sqldf

In [2]:
# Load Clients dataFrame:
clients = pd.read_csv('Database/Clients.csv')

# Load Payments dataFrame:
payments = pd.read_csv('Database/Payments.csv')

### SQL

In [4]:


query='''

-- Clean duplicated client_id (For this exercise, we will use the first entry (oldest duplicated entry))

WITH cleanedClients AS(
SELECT 
    client_id,
    entity_type, 
    MIN(entity_year_established) AS entity_year_established
    
FROM clients

GROUP BY client_id
),

--########## Payment and left join clients #######
-- Merge payments and convert date to get year:

merged AS(
SELECT 
    *, 
    strftime('%Y', datetime(transaction_date, 'unixepoch')) AS year
    
FROM payments

LEFT JOIN cleanedClients ON cleanedClients.client_id = payments.client_id

WHERE payment_code = 'PAYMENT'
),

-- ############### 2018 ####################
-- GET 2018 RANKED by total_payment_amt

get2018_summedPayments AS(
SELECT 
    *, 
    SUM(payment_amt) AS total_payment_2018

FROM merged

WHERE merged.year == '2018'

GROUP BY client_id

ORDER BY total_payment_2018 DESC
),

ranked_2018 AS(
SELECT 
    *,
    ROW_NUMBER() OVER (ORDER BY total_payment_2018 DESC) AS rank_per_payment_2018
FROM get2018_summedPayments
),
-- ############################################

-- ########### OVERALL RANKED PER ENTITY TYPE ########################
getOverall_summedPayments as(
SELECT 
    *, 
    SUM(payment_amt) AS total_payment_Overall

FROM merged

GROUP BY client_id

ORDER BY total_payment_Overall desc
),

ranked_Overall_perEntity AS(
SELECT 
    *,
    RANK() OVER (PARTITION BY entity_type ORDER BY total_payment_Overall DESC) AS rank_per_entity_overall
    
FROM getOverall_summedPayments
)

-- #############################################
-- #############################################

-- Joins to make required dataset:

SELECT 
    ranked_2018.rank_per_payment_2018, 
    cleanedClients.client_id, 
    cleanedClients.entity_type,  
    ranked_2018.total_payment_2018, 
    ranked_Overall_perEntity.rank_per_entity_overall

FROM cleanedClients

LEFT JOIN ranked_2018 ON ranked_2018.client_id = cleanedClients.client_id
LEFT JOIN ranked_Overall_perEntity ON ranked_Overall_perEntity.client_id = cleanedClients.client_id

ORDER BY ranked_2018.total_payment_2018 DESC

LIMIT 20
'''



sqldf(query, env=None).to_csv('challenge_response.csv',index=False)